# Data Analysis Examples

Now that we've reached th end of the book's main chapters, we're going to take a look at a number of real-world datasets. For each dataset, we'll use the techniques presented in this book to extract  meaning from the raw data. The demonstrated techniques can be applied to all manner of the other datasets, including your own. This chapter contains a collection of miscellaneous example datasets that you can use for practice with the tools ion this book.
The examples datasets are found in the book's accompanying GitHub repository (http://github.com/wesm/pydata-book)

## USA.gov Data from Bitly

In [ ]:
path = '../datasets/bitly_usagov/example.txt'

In [ ]:
open(path).readline()

In [ ]:
import json
path = '../datasets/bitly_usagov/example.txt'
records = [json.loads(line) for line in open(path)]

In [ ]:
records[0]

### Counting Time Zones in Pure Python

In [ ]:
# time_zones = [rec['tz'] for rec in records]

In [ ]:
time_zones = [rec['tz'] for rec in records if 'tz' in rec]

In [ ]:
time_zones[:10]

In [ ]:
def get_counts(sequence):
    counts = {}
    for x in sequence:
        if x in counts:
            counts[x] += 1
        else: 
            counts[x] = 1
    return counts

In [ ]:
from collections import defaultdict

In [ ]:
def get_counts2(sequence):
    counts = defaultdict(int) # values will initialize to 0
    for x in sequence: 
        counts[x] += 1
    return counts

In [ ]:
counts = get_counts(time_zones)

In [ ]:
counts['America/New_York']

In [ ]:
len(time_zones)

In [ ]:
def top_counts(count_dict,n=10):
    value_key_pairs = [(count, tz) for tz, count in count_dict.items()]
    value_key_pairs.sort()
    return value_key_pairs[-n:]

In [ ]:
top_counts(counts)

In [ ]:
from collections import Counter

In [ ]:
counts = Counter(time_zones)

In [ ]:
counts.most_common(10)

### Counting Time Zones with pandas

In [ ]:
import pandas as pd

In [ ]:
frame = pd.DataFrame(records)

In [ ]:
frame.info()

In [ ]:
frame['tz'][:10]

In [ ]:
tz_counts = frame['tz'].value_counts()

In [ ]:
tz_counts[:10]

In [ ]:
import seaborn as sns

In [ ]:
subset = tz_counts[:10]

In [ ]:
sns.barplot(y=subset.index,x=subset.values)

In [ ]:
frame['a'][1]

In [ ]:
frame['a'][50]

In [ ]:
frame['a'][51][:50]

In [ ]:
results = pd.Series([x.split()[0] for x in frame.a.dropna()])

In [ ]:
results[:5]

In [ ]:
results.value_counts()[:8]

In [ ]:
cframe = frame[frame.a.notnull()]

In [ ]:
import numpy as np

In [ ]:
cframe['os'] = np.where(cframe['a'].str.contains('Windows'),
                       'Windows','Not Windows')

In [ ]:
cframe['os'][:5]

In [ ]:
by_tz_os = cframe.groupby(['tz','os'])

In [ ]:
agg_counts = by_tz_os.size().unstack().fillna(0)

In [ ]:
agg_counts[:10]

In [ ]:
# Use to sort in ascending order
indexer = agg_counts.sum(1).argsort()

In [ ]:
indexer[:10]

In [ ]:
count_subset = agg_counts.take(indexer[-10:])

In [ ]:
count_subset

In [ ]:
agg_counts.sum(1).nlargest(10)

In [ ]:
# Rearrange the data for plotting
count_subset = count_subset.stack()

In [ ]:
count_subset.name = 'total'

In [ ]:
count_subset = count_subset.reset_index()

In [ ]:
count_subset[:10]

In [ ]:
sns.barplot(x='total',y='tz',hue='os',data=count_subset)

In [ ]:
def norm_total(group):
    group['normed_total']=group.total/group.total.sum()
    return group

In [ ]:
results = count_subset.groupby('tz').apply(norm_total)

In [ ]:
sns.barplot(x='normed_total',y='tz',hue='os',data=results)

In [ ]:
g = count_subset.groupby('tz')

In [ ]:
results2 = count_subset.total / g.total.transform('sum')

## MovieLens 1M Dataset

In [ ]:
import pandas as pd

In [ ]:
# Make display smaller
pd.options.display.max_rows = 10

In [ ]:
unames = ['user_id','gender','age','occupation','zip']
users = pd.read_table('../datasets/movielens/users.dat',sep='::',
                     header=None, names=unames)

In [ ]:
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_table('../datasets/movielens/ratings.dat',sep='::',
                       header=None, names=rnames)

In [ ]:
mnames = ['movie_id','title','genres']
movies = pd.read_table('../datasets/movielens/movies.dat', sep='::',
                      header=None, names=mnames)

In [ ]:
users[:5]

In [ ]:
ratings[:5]

In [ ]:
movies[:5]

In [ ]:
ratings

In [ ]:
data = pd.merge(pd.merge(ratings,users),movies)

In [ ]:
data

In [ ]:
data.iloc[0]

In [ ]:
mean_ratings = data.pivot_table('rating',index='title',
                                columns='gender', aggfunc='mean')

In [ ]:
mean_ratings[:5]

In [ ]:
ratings_by_title = data.groupby('title').size()

In [ ]:
ratings_by_title[:10]

In [ ]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]

In [ ]:
active_titles

In [ ]:
# Select rows on the index
mean_ratings = mean_ratings.loc[active_titles]

In [ ]:
mean_ratings

In [ ]:
top_female_ratings = mean_ratings.sort_values(by='F',ascending=False)

In [ ]:
top_female_ratings[:10]

### Measuring Rating Disagreement

In [ ]:
mean_ratings['diff'] = mean_ratings['M']-mean_ratings['F']

In [ ]:
sorted_by_diff = mean_ratings.sort_values(by='diff')

In [ ]:
sorted_by_diff[:10]

In [ ]:
# Reverse order of rows, take first 10 rows
sorted_by_diff[::-1][:10]

In [ ]:
#Standard deviation of rating grouped by title
rating_std_by_title = data.groupby('title')['rating'].std()

In [ ]:
# Filter down to active_titles
rating_std_by_title = rating_std_by_title.loc[active_titles]

In [ ]:
# Order Series by value in descending order
rating_std_by_title.sort_values(ascending=False)[:10]

## US Baby Names 1880-2010

In [ ]:
!head -n 10 ../datasets/babynames/yob1880.txt

In [ ]:
import pandas as pd

In [ ]:
names1880 = pd.read_csv('../datasets/babynames/yob1880.txt',
                       names=['name','sex','births'])

In [ ]:
names1880

In [ ]:
names1880.groupby('sex').births.sum()

In [ ]:
years = range(1880,2011)
pieces = []
columns = ['names','sex','births']

In [ ]:
for year in years:
    path = '../datasets/babynames/yob%d.txt' % year
    frame = pd.read_csv(path,names=columns)
    
    frame['year'] = year
    pieces.append(frame)

In [ ]:
# Concatenate everything into a single DataFrame
names = pd.concat(pieces, ignore_index=True)

In [ ]:
names

In [ ]:
total_births = names.pivot_table('births',index='year',
                                 columns='sex',aggfunc=sum)

In [ ]:
total_births.tail()

In [ ]:
total_births.plot(title='Total births by sex and year')

In [ ]:
def add_prop(group):
    group['prop']= group.births / group.births.sum()
    return group
names = names.groupby(['year','sex']).apply(add_prop)

In [ ]:
names

In [ ]:
names.groupby(['year','sex']).prop.sum()

In [ ]:
def get_top1000(group):
    return group.sort_values(by='births',ascending=False)[:1000]
grouped = names.groupby(['year','sex'])
top1000 = grouped.apply(get_top1000)
# Drop the group index, not needed
top1000.reset_index(inplace=True,drop=True)

In [ ]:
pieces = []
for year, group in names.groupby(['year','sex']):
    pieces.append(group.sort_values(by='births',ascending=False)[:1000])
top1000 = pd.concat(pieces, ignore_index = True)

In [ ]:
top1000

### Analyzing Naming Trends

In [ ]:
boys = top1000[top1000.sex == 'M']

In [ ]:
girls = top1000[top1000.sex == 'F']

In [ ]:
total_births = top1000.pivot_table('births', index='year',
                                   columns='names',
                                   aggfunc=sum)

In [ ]:
total_births.info()

In [ ]:
subset = total_births[['John','Harry','Mary','Marilyn']]

In [ ]:
subset.plot(subplots=True,figsize=(12,10),grid=False,
           title="Number of births per year")

#### Measuring the increase in naming diversity

In [ ]:
table = top1000.pivot_table('prop', index = 'year',
                           columns = 'sex', aggfunc=sum)

In [ ]:
table.plot(title='Sum of table1000.prop by year and sex',
          yticks=np.linspace(0,1.2,13), xticks=range(1880,2020,10))

In [ ]:
df = boys[boys.year==2010]

In [ ]:
df

In [ ]:
prop_cumsum = df.sort_values(by='prop',ascending=False).prop.cumsum()

In [ ]:
prop_cumsum[:10]

In [ ]:
prop_cumsum.values.searchsorted(0.5)

In [ ]:
df = boys[boys.year==1900]

In [ ]:
in1900 = df.sort_values(by='prop',ascending=False).prop.cumsum()

In [ ]:
in1900.values.searchsorted(0.5)+1

## USDA Food Database

## 2012 Federal Election Commission Database

### Donation Statistics by Occupation and Employer

### Bucketing Donation Amounts

### Donation Statistics by State